In [ ]:
#Installerer de nødvendige pakkene

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
#Laster inn filen som omhandler det gitte selskapet

AppleR = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/6 Big/Apple_Ferdig.csv")

In [ ]:
AppleR.shape

In [ ]:
#Ser hvordan datasettet ser ut

AppleR.head(3)

In [ ]:
#Laster inn aksjeprisen for det gitte selskapet

AppleAksje = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Apple/AppleStockPrices_Label.csv")

In [ ]:
AppleAksje.shape

In [ ]:
#Ser hvordan datasettet ser ut

AppleAksje.head(3)

In [ ]:
#Merger datasettene sammen basert på dato

merge = AppleR.merge(AppleAksje, how = "inner", on = "Date")

In [ ]:
#Installerer pakker som er nødvendige

import nltk
import ssl
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')

In [ ]:
#Lager flere funksjoner som gir ut sentiment scoren til SentiWordNet på artikkel innholdet

pos=neg=obj=count=0

postagging = []

for article in merge["article_content"]:
    list = word_tokenize(article)
    postagging.append(nltk.pos_tag(list))

merge["pos_tags"] = postagging

def penn_to_wn(tag):
    if tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("R"):
        return wn.ADV
    elif tag.startswith("V"):
        return wn.VERB
    return None


#Returnerer liste av positive / negative og objektive score,
#men returnerer en tom liste dersom innholdet ikke er i SentiWordNet

def get_sentiment(word, tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Synset er en spesiell type av et enkelt grensesnitt som er i NLTK for å kunne finne tak i ordene som er i WordNet. 
    #Når det forekommer Synset er det en gruppering av synonyme ord som uttrykker det samme konseptet. 
    #Noen av ordene har bare en Synset, mens noen har flere.
    
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    #Velger det første og mest vanlige uttrykket.
    
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    pos=neg=obj=count=0
    
    ###################################################################################
senti_score = []

for pos_val in merge["pos_tags"]:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #Positiv score er plassert i andre posisjon.
            neg = neg + score[2]  #Negativ score er plassert i tredje posisjon.
        except:
            continue
    senti_score.append(pos - neg)
    pos=neg=0    
    
merge["senti_score"] = senti_score
print(merge["senti_score"])

#Ser hvordan datasettet ser ut etter at sentiment scoren er lagt til

print(merge.head)

In [ ]:
#Lager en ny kolonne som innholder "Positive", "Neutral" og "Negative" i forhold til visse restriksjoner.

overall=[]
for i in range(len(merge)):
    if merge["senti_score"][i]>= 0.05:
        overall.append("Positive")
    elif merge["senti_score"][i]<= -0.05:
        overall.append("Negative")
    else:
        overall.append("Neutral")
merge["Overall Sentiment"] = overall

In [ ]:
#Ser hvordan datasettet ser ut etter siste tillegget

merge.head(3)

In [ ]:
#Beholder de kolonnene som er nødvendige i en ny liste

keep_columns = ["Open", "Low", "High", "Date", "Close", "Volume", "senti_score", "Overall Sentiment", "Label"]
df = merge[keep_columns]
df

In [ ]:
#Gjør om datasettet til en csv fil før den går gjennom filtrering i R,
#og går gjennom samme prosess som Vader og TextBlob med klassifiseringene.